In [ ]:
import asyncio
import aiohttp
from asyncio_throttle import Throttler
import os
import dotenv
import pandas as pd
import logging

In [ ]:

logger = logging.getLogger(__name__)

file_handler = logging.FileHandler('log.txt')
file_handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s'))

logger.addHandler(file_handler)

In [ ]:
%load_ext dotenv
%dotenv keys.env


api_key = os.getenv('bibsKey')


headers = {
    "Authorization": f"apikey {api_key}",
    "Accept": "application/json",
}


In [ ]:
# made to work off of report output from the physical items subject area in analytics.
# required columns are Holding Id, MMS ID, and Physical Item ID (PID).

bibs = pd.read_csv('bibs.csv')  

In [ ]:
async def gather_tasks(urls, headers=None):
    """Makes a GET call to each URL in the specified list, then makes a PUT call to the same URL after changing something about the object that was received in the GET."""
    throttler = Throttler(rate_limit=20)
    # Create a list of tasks
    tasks = []
    for url in urls:
        task = asyncio.create_task(make_calls(url, headers=headers, throttler=throttler))
        tasks.append(task)

    # Wait for all of the tasks to finish
    await asyncio.gather(*tasks)


In [ ]:
async def make_calls(url, headers=None, throttler=None):
    """Makes a GET call to the specified URL, then makes a PUT call to the same URL after changing something about the object that was received in the GET."""

    async with throttler:
        # Make the GET call
        try:
            async with aiohttp.ClientSession() as session:
                response = await session.get(url, headers=headers)

                # pull item object
                object = await response.json()

                # Map inventory price to replacement cost

                inv_cost = object['item_data']["inventory_price"]

                object['item_data']["replacement_cost"] = inv_cost

                title = object['bib_data']['title']
                barcode = object['item_data']['barcode']


            put_headers = {
                'Content-Type': 'application/json',
                'Authorization': f'apikey {api_key}',

            }
            # Make the PUT call
            async with throttler:
                async with aiohttp.ClientSession() as session:
                    put_response = await session.put(url, json=object, headers=put_headers)
            if put_response.status == 200:
                print(f'Updated {title} and {barcode}. Inventory cost was {inv_cost}, replacement cost is now {object["item_data"]["replacement_cost"]}.')

        except Exception as e:
            logger.error(f'{e} for {url}')

In [ ]:
# Creates the URL endpoints needed to make citation requests.
def build_url(df):  

    a = []


    for index, row in df.iterrows():
        mms_id, holding_id, item_pid = (
            row["MMS Id"],
            row["Holding Id"],
            row["Physical Item Id"],
        )

        url = f"https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}"

        a.append(url)

    return a

In [ ]:
urls = build_url(bibs)

In [ ]:
await gather_tasks(urls, headers=headers)